This notebook is a guide for running a Concordia Framework Simulation

### Language Model setup

In [2]:
# @title Imports
import concordia.prefabs.simulation.questionnaire_simulation as questionnaire_simulation
import concordia.prefabs.entity as entity_prefabs
import concordia.prefabs.game_master as game_master_prefabs

from concordia.type_checks import prefab as prefab_lib
from concordia.language_model.model_client_initialization import ModelClient
from concordia.utils import helper_functions

from IPython import display


AttributeError: module 'concordia.type_checks.entity_component' has no attribute 'Component'

In [ ]:
# Initialize the client
client = ModelClient()
model = client.model
embedder = client.embedder

In [ ]:
test = model.sample_text(
    'Answer with a single statement: What is the capital of France?')
print(test)

# Parameters

In [4]:
# Load prefabs from packages to make the specific palette to use here.

prefabs = {
    **helper_functions.get_package_classes(entity_prefabs),
    **helper_functions.get_package_classes(game_master_prefabs),
}

In [ ]:
display.display(
    display.Markdown(helper_functions.print_pretty_prefabs(prefabs)))

In [5]:
from concordia.contrib.data.questionnaires import depression_stress_anxiety_scale

DASS = depression_stress_anxiety_scale.DASSQuestionnaire()


In [9]:
lord_of_the_rings_characters = [
    "Frodo Baggins",
    "Samwise Gamgee",
    "Meriadoc Brandybuck",
    "Peregrin Took",
    "Gandalf",
    "Aragorn",
    "Legolas",
    "Gimli",
    "Boromir",
    "Sauron",
    "Saruman",
    "Gollum",
    "Arwen",
    "Galadriel",
    "Elrond",
    "Théoden",
    "Éowyn",
    "Faramir",
    "Denethor",
    "Bilbo Baggins"
]

In [6]:
# Shoerter list for testing
lord_of_the_rings_characters = [
    "Frodo Baggins",
    "Sauron"
]

In [7]:
# Scene 1 - Frodo in the Shire

instances = []

for name in lord_of_the_rings_characters:
  instances.append(prefab_lib.InstanceConfig(
      prefab='basic__Entity',
      role=prefab_lib.Role.ENTITY,
      params={
          'name': name,
      },
  ))

instances.append(prefab_lib.InstanceConfig(
        prefab='interviewer__GameMaster',
        role=prefab_lib.Role.GAME_MASTER,
        params={
            'name': 'interviewer',
            'player_names': lord_of_the_rings_characters,
            'questionnaires': [DASS],
            'verbose': False,
            'embedder': embedder,
        },
    ),

)

In [8]:
config = prefab_lib.Config(
    default_premise=f"""Under the oppressive gaze of Sauron's searching Eye, a weary Frodo inches toward Mount Doom's fires, the Ring's weight a constant burden as the world's fate hangs in the balance.""",
    default_max_steps=1,
    prefabs=prefabs,
    instances=instances,
)

In [9]:
# @title Initialize the simulation
runnable_simulation = questionnaire_simulation.QuestionnaireSimulation(
    config=config,
    model=model,
    embedder=embedder,
)

In [10]:
# @title Run the simulation
results_log = runnable_simulation.play(max_steps=1)

In [14]:
questionnaire_comp = runnable_simulation.game_masters[0].get_component(
    'questionnaire'
)
questionnaire_result_df = questionnaire_comp.get_questionnaires_results()
print(questionnaire_result_df)

                anxiety  depression    stress
Frodo Baggins  1.857143    1.285714  1.714286
Sauron         2.428571    1.785714  2.071429


In [ ]:
print(questionnaire_comp.get_answers())